In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "HuggingFaceTB/SmolLM-135M-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16, attn_implementation='sdpa').cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import transformers
print(transformers.__version__)

4.37.2


In [4]:
sample_text = open("snapkv.txt", "r").read()
encoded_tokens = tokenizer(sample_text, return_tensors="pt")
for key in encoded_tokens:
    encoded_tokens[key] = encoded_tokens[key].cuda()
print(encoded_tokens.input_ids.shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (17256 > 2048). Running this sequence through the model will result in indexing errors


torch.Size([1, 17256])


In [5]:
query_window = 48
template_window = 48
max_tokens = 2048

In [6]:
import torch
import time
import torch.nn.functional as F
import torch.nn as nn
import math

# perform qk calculation and get indices
# this version will not update in inference mode
class SnapKVCluster():
    def __init__(self, query_window = 64, template_window = 64, max_capacity_prompt = 256 + 64, kernel_size = 5, pooling = 'avgpool'):
        self.query_window = query_window
        self.template_window = template_window
        self.max_capacity_prompt = max_capacity_prompt
        assert self.max_capacity_prompt - self.query_window > 0
        self.kernel_size = kernel_size
        self.pooling = pooling

    def update_kv(self, key_states, query_states, value_states, attention_mask, num_key_value_groups):
        # check if prefix phase
        assert key_states.shape[-2] == query_states.shape[-2]
        bsz, num_heads, q_len, head_dim = query_states.shape
        if q_len < self.max_capacity_prompt:
            return key_states, value_states
        else:
            attn_weights = torch.matmul(query_states[..., -self.query_window:, :], key_states.transpose(2, 3)) / math.sqrt(head_dim)
            mask = torch.full((self.query_window, self.query_window), torch.finfo(attn_weights.dtype).min, device=attn_weights.device)
            mask_cond = torch.arange(mask.size(-1), device=attn_weights.device)
            mask.masked_fill_(mask_cond < (mask_cond + 1).view(mask.size(-1), 1), 0)
            mask = mask.to(attn_weights.device)
            attention_mask = mask[None, None, :, :]

            attn_weights[:, :, -self.query_window:, -self.query_window:] += attention_mask
            attn_weights = torch.where(torch.isinf(attn_weights), torch.tensor(-1e9, dtype=attn_weights.dtype), attn_weights)
            # print("Weights before softmax: ", attn_weights)
            attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
            attn_weights_sum = attn_weights[:, :, -self.query_window:, :-self.query_window].sum(dim = -2)
            if self.pooling == 'avgpool':
                attn_cache = F.avg_pool1d(attn_weights_sum, kernel_size = self.kernel_size, padding=self.kernel_size//2, stride=1)
            elif self.pooling == 'maxpool':
                attn_cache = F.max_pool1d(attn_weights_sum, kernel_size = self.kernel_size, padding=self.kernel_size//2, stride=1)
            else:
                raise ValueError('Pooling method not supported')
            return attn_cache

snapkv_cluster = SnapKVCluster(query_window=query_window, template_window=template_window, max_capacity_prompt=max_tokens)

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576, padding_idx=2)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [8]:
from typing import Optional, Tuple
from transformers.cache_utils import Cache
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb, repeat_kv

def sdpa_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    # print("Running custom forward function")
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

    if past_key_value is not None:
        cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    if attention_mask is not None:
        if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )

    # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
    # Reference: https://github.com/pytorch/pytorch/issues/112577.
    if query_states.device.type == "cuda" and attention_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    attn_output = torch.nn.functional.scaled_dot_product_attention(
        query_states,
        key_states,
        value_states,
        attn_mask=attention_mask,
        dropout_p=self.attention_dropout if self.training else 0.0,
        # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
        is_causal=self.is_causal and attention_mask is None and q_len > 1,
    )

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size)

    attn_output = self.o_proj(attn_output)

    attn_cache = snapkv_cluster.update_kv(key_states, query_states, value_states, attention_mask, self.num_key_value_groups)

    return attn_output, attn_cache, past_key_value

for i in range(len(model.model.layers)):
    model.model.layers[i].self_attn.forward = sdpa_forward.__get__(model.model.layers[i].self_attn, type(model.model.layers[i].self_attn))

In [9]:
from einops import repeat
import time

start_time = time.perf_counter()
top_tokens = []

encoded_tokens = tokenizer.encode(sample_text, return_tensors="pt")[0]  
print("Encoding tokens took: ", time.perf_counter() - start_time)
print("Encoded tokens shape: ", encoded_tokens.shape)

chunk_count = math.ceil((encoded_tokens.shape[-1] - template_window - query_window) / (max_tokens - query_window - template_window))
print(encoded_tokens[template_window:-query_window])
split_tensors = encoded_tokens[template_window:-query_window].chunk(chunks=chunk_count, dim=-1)

last_chunk_size = split_tensors[-1].shape[-1]
print("Last chunk size: ", last_chunk_size)

# split_tensors = torch.split(encoded_tokens[template_adjustment:-query_window], max_tokens - query_window - template_adjustment)
max_split_tokens = 0
for split_tensor in split_tensors:
    max_split_tokens = max(max_split_tokens, split_tensor.shape[-1])
padded_tensors = []
attention_masks = []

template_tokens = encoded_tokens[:template_window]
query_tokens = encoded_tokens[-query_window:]

print("Query tokens shape: ", query_tokens.shape)
print("Chunks shape: ", [t.shape for t in split_tensors])
print("Template tokens shape: ", template_tokens.shape)

max_tokens_length = split_tensors[0].shape[-1] + template_window + query_window

for i, chunk_tensor in enumerate(split_tensors):
    joined_tensor = torch.cat((template_tokens, chunk_tensor, query_tokens), dim=-1) # -> take care of template tensor later
    pad_tensor = torch.tensor(tokenizer.pad_token_id).expand(max_tokens - joined_tensor.shape[-1])
    chunk_attention_mask = torch.IntTensor([0] * (max_tokens - chunk_tensor.shape[-1]) + [1] * (chunk_tensor.shape[-1]))
    padded_tensors.append(torch.cat((pad_tensor, joined_tensor), dim=-1))
    attention_masks.append(chunk_attention_mask)

attention_masks = torch.stack(attention_masks, dim=0)
padded_tensors = torch.stack(padded_tensors, dim=0)
print("Generated padded tensors: ", padded_tensors.shape, " and attention masks: ", attention_masks.shape, " in ", time.perf_counter() - start_time)

padded_tensors = padded_tensors.to(model.device)
attention_masks = attention_masks.to(model.device)

# batch run 
outputs = model(padded_tensors, attention_mask=attention_masks, output_attentions=True)#, return_dict_in_generate=True, output_attentions=True)
for key in outputs:
    print("Output key: ", key)
print("Forward pass completed in: ", time.perf_counter() - start_time)

print("Output attentions: ", outputs.attentions)
print("First dim length: ", len(outputs.attentions))
for x in outputs.attentions:
    print("Sublen: ", len(x))

summed_weights = torch.stack(outputs.attentions, dim=0).sum((0, 2))
print("original summed weights shape: ", summed_weights.shape)

# Isolate the final tensor
# TODO: do the math to debug this...
# print("Starting template window chunk size: ", summed_weights[0, :template_window].shape)
# print("Main portion: ", summed_weights[:-1, template_window:].flatten().shape)
# print("End portion: ", summed_weights[-1, -last_chunk_size:].shape)

first_chunk_size = split_tensors[0].shape[-1]
flat_weights = summed_weights[0, - first_chunk_size - template_window : - first_chunk_size]


for split_chunk, attn_weight_chunk in zip(split_tensors, summed_weights):
    print("Flat weights shape: ", flat_weights.shape)
    print("Attention weight chunk: ", attn_weight_chunk[-split_chunk.shape[-1]:].shape)
    flat_weights = torch.cat((flat_weights, attn_weight_chunk[-split_chunk.shape[-1]:]), dim=-1)

# Adding the template, the summed weights in the middle, and only the relevant portion at the end

# summed_weights = torch.stack(summed_weights, dim=0)
# summed_weights = summed_weights.sum(dim=0)
print("Flat weights shape: ", flat_weights.shape)
print(flat_weights)

indices = torch.sort(flat_weights.topk(max_tokens * 2, dim=-1).indices.cpu()).values
print(indices)
selected_tokens = encoded_tokens[indices]
decoded_text = tokenizer.decode(selected_tokens)
print("Decoded text: ", decoded_text)

# get the top tokens attended to by the query tokens
# create one single list of "most important tokens"
# return this string.
end_time = time.perf_counter()
print("Time taken: ", end_time - start_time)

Encoding tokens took:  0.019971504982095212
Encoded tokens shape:  torch.Size([17256])
tensor([   70, 13842,   281,  ...,  2526,  5605, 19926])
Last chunk size:  1904
Query tokens shape:  torch.Size([48])
Chunks shape:  [torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1907]), torch.Size([1904])]
Template tokens shape:  torch.Size([48])
Generated padded tensors:  torch.Size([9, 2048])  and attention masks:  torch.Size([9, 2048])  in  0.02226422500098124


Output key:  logits
Output key:  past_key_values
Output key:  attentions
Forward pass completed in:  0.6381797170033678
Output attentions:  (tensor([[[0.0324, 0.0432, 0.0540,  ..., 0.1277, 0.0955, 0.0791],
         [0.0220, 0.0291, 0.0361,  ..., 0.3423, 0.2666, 0.2192],
         [0.0331, 0.0442, 0.0553,  ..., 0.1398, 0.0977, 0.0832],
         ...,
         [0.0299, 0.0401, 0.0504,  ..., 0.0488, 0.0367, 0.0331],
         [0.0175, 0.0239, 0.0304,  ..., 0.2347, 0.2291, 0.1967],
         [0.0235, 0.0313, 0.0389,  ..., 0.0786, 0.0652, 0.0571]],

        [[0.0327, 0.0435, 0.0544,  ..., 0.1050, 0.0772, 0.0588],
         [0.0225, 0.0297, 0.0369,  ..., 0.2961, 0.2368, 0.1899],
         [0.0338, 0.0451, 0.0564,  ..., 0.1066, 0.0730, 0.0557],
         ...,
         [0.0302, 0.0406, 0.0510,  ..., 0.0500, 0.0373, 0.0290],
         [0.0183, 0.0251, 0.0320,  ..., 0.0343, 0.0284, 0.0217],
         [0.0242, 0.0323, 0.0401,  ..., 0.0793, 0.0619, 0.0465]],

        [[0.0331, 0.0442, 0.0552,  ..., 0.1338,